In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from itertools import combinations
import missingno as msno
import matplotlib.ticker as mticker # NameError 해결을 위한 Ticker 임포트

try:
    plt.rcParams['font.family'] = 'Malgun Gothic'
except:
    plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False


In [2]:
def downcast(df: pd.DataFrame, verbose: bool = True) -> pd.DataFrame:

    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        dtype_name = df[col].dtype.name
        
        if dtype_name == 'object':
            if df[col].nunique() / df[col].shape[0] < 0.5:
                df[col] = df[col].astype('category')
        
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
            
        elif dtype_name.startswith('int'):
            df[col] = pd.to_numeric(df[col], downcast='integer')
            
        elif dtype_name.startswith('float'):
            if (df[col].round() == df[col]).all():
                 df[col] = pd.to_numeric(df[col], downcast='integer')
            else:
                 df[col] = pd.to_numeric(df[col], downcast='float')
                 
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f"메모리 사용량: {start_mem:.2f} MB -> {end_mem:.2f} MB")
        print(f"감소율: {(start_mem - end_mem) / start_mem * 100:.1f}%")
        
    return df

In [3]:
train_data = pd.read_csv("data/output/rawdata/train.csv")
train_data['날짜'] = pd.to_datetime(train_data['날짜'])
train_data = downcast(train_data)

test_data = pd.read_csv("data/output/rawdata/test.csv")
test_data = downcast(test_data)

메모리 사용량: 223.99 MB -> 76.27 MB
감소율: 65.9%
메모리 사용량: 13.07 MB -> 2.82 MB
감소율: 78.4%
메모리 사용량: 13.07 MB -> 2.82 MB
감소율: 78.4%


### 중복제거

In [4]:
def handle_duplicates(df):
    """
    데이터프레임의 중복 행을 찾아 출력하고, 첫 번째 행만 남기고 제거합니다.
    
    Args:
        df (pd.DataFrame): 처리할 데이터프레임
        
    Returns:
        pd.DataFrame: 중복이 제거된 데이터프레임
    """
    original_rows = len(df)
    duplicates = df[df.duplicated(keep=False)]

    if not duplicates.empty:
        print(f"총 **{len(duplicates):,}개**({duplicates.duplicated().sum():,}쌍)의 중복 행이 발견되었습니다.")
        print("\n**[중복 데이터 목록]**")
        print("```")
        print(duplicates.sort_values(by=list(df.columns)).to_string())
        print("```")
        
        print("\n중복된 행 중 첫 번째 행만 남기고 모두 제거합니다...")
        processed_df = df.drop_duplicates(keep='first')
        new_rows = len(processed_df)
        print(f"✅ **제거 완료.** 전체 행 수가 `{original_rows:,}`개에서 `{new_rows:,}`개로 변경되었습니다.")
        
        return processed_df
        
    else:
        print("✅ 중복된 행이 없습니다.")
        return df

In [5]:
train_data = handle_duplicates(train_data)

총 **12개**(6쌍)의 중복 행이 발견되었습니다.

**[중복 데이터 목록]**
```
                날짜  월ID  상점ID   상품ID    판매가  판매량                     상점명                                              상품명  상품분류ID            상품분류명
76961   2013-01-05    0    54  20130  149.0    1         Химки ТЦ "Мега"                          УЧЕНИК ЧАРОДЕЯ (регион)      40       Кино - DVD
76962   2013-01-05    0    54  20130  149.0    1         Химки ТЦ "Мега"                          УЧЕНИК ЧАРОДЕЯ (регион)      40       Кино - DVD
1435365 2014-02-23   13    50   3423  999.0    1      Тюмень ТЦ "Гудвин"  Far Cry 3 (Classics) [Xbox 360, русская версия]      23  Игры - XBOX 360
1435367 2014-02-23   13    50   3423  999.0    1      Тюмень ТЦ "Гудвин"  Far Cry 3 (Classics) [Xbox 360, русская версия]      23  Игры - XBOX 360
1496765 2014-03-23   14    21   3423  999.0    1  Москва МТРЦ "Афи Молл"  Far Cry 3 (Classics) [Xbox 360, русская версия]      23  Игры - XBOX 360
1496766 2014-03-23   14    21   3423  999.0    1  Москва МТРЦ "Афи 

### 이상치 제거 (판매가 6000이상, 판매량 5이상)

In [6]:
# 이상치 조건에 해당하는 데이터 확인
outliers = train_data[(train_data['판매가'] >= 6000) | (train_data['판매량'] >= 5)]

print(f"이상치 데이터 수: {len(outliers):,}개")
print("\n[이상치 데이터 샘플 (처음 10개)]")
print(outliers.head(10))

이상치 데이터 수: 66,091개

[이상치 데이터 샘플 (처음 10개)]
             날짜  월ID  상점ID  상품ID      판매가  판매량                  상점명  \
252  2013-01-02    0    25  3432    599.0    5  Москва ТРК "Атриум"   
352  2013-01-15    0    25  2973   2499.0   13  Москва ТРК "Атриум"   
353  2013-01-16    0    25  2973   2499.0    5  Москва ТРК "Атриум"   
404  2013-01-25    0    25  2972    599.0   13  Москва ТРК "Атриум"   
405  2013-01-26    0    25  2972    599.0    5  Москва ТРК "Атриум"   
421  2013-01-26    0    25  3186    419.3    5  Москва ТРК "Атриум"   
1066 2013-01-18    0    25  4861   8490.0    1  Москва ТРК "Атриум"   
1203 2013-01-20    0    25  5613   6190.0    1  Москва ТРК "Атриум"   
1617 2013-01-03    0    25  3321   1999.0    5  Москва ТРК "Атриум"   
1701 2013-01-03    0    25  4384  13499.0    1  Москва ТРК "Атриум"   

                                                    상품명  상품분류ID  \
252               Far Cry 3 [PC, Jewel, русская версия]      30   
352           DmC Devil May Cry [PS3, русс

In [7]:
# 이상치 제거
original_rows = len(train_data)
train_data = train_data[~((train_data['판매가'] >= 6000) | (train_data['판매량'] >= 5))]

# 처리 결과 출력
print(f"이상치 제거 완료:")
print(f"- 전체 행 수: {original_rows:,}개 → {len(train_data):,}개")
print(f"- 제거된 행 수: {original_rows - len(train_data):,}개")
print(f"- 제거 비율: {(original_rows - len(train_data))/original_rows*100:.2f}%")

# 처리 후 통계 확인
print("\n[처리 후 판매가/판매량 통계]")
print(train_data[['판매가', '판매량']].describe([.01, .05, .25, .5, .75, .95, .99]))

이상치 제거 완료:
- 전체 행 수: 2,935,843개 → 2,869,752개
- 제거된 행 수: 66,091개
- 제거 비율: 2.25%

[처리 후 판매가/판매량 통계]
                판매가           판매량
count  2.869752e+06  2.869752e+06
mean   7.524252e+02  1.114773e+00
std    8.088992e+02  4.364864e-01
min   -1.000000e+00 -2.200000e+01
1%     2.800000e+01  1.000000e+00
5%     9.934000e+01  1.000000e+00
25%    2.490000e+02  1.000000e+00
50%    3.990000e+02  1.000000e+00
75%    9.900000e+02  1.000000e+00
95%    2.599000e+03  2.000000e+00
99%    3.598000e+03  3.000000e+00
max    5.999500e+03  4.000000e+00


### 상점명 유사도 비교 후 처리

In [8]:
train_data["상점ID"].unique()

array([59, 25, 24, 23, 19, 22, 18, 21, 28, 27, 29, 26,  4,  6,  2,  3,  7,
        0,  1, 16, 15,  8, 10, 14, 13, 12, 53, 31, 30, 32, 35, 56, 54, 47,
       50, 42, 43, 52, 51, 41, 38, 44, 37, 46, 45,  5, 57, 58, 55, 17,  9,
       49, 39, 40, 48, 34, 33, 20, 11, 36], dtype=int8)

In [9]:
from difflib import SequenceMatcher

def calculate_similarity(str1, str2):
    """두 문자열 간의 유사도를 계산"""
    return SequenceMatcher(None, str1, str2).ratio()

def find_similar_shop_names(df, threshold=0.8):
    """
    상점명 간의 유사도를 계산하고 유사한 쌍을 찾아냅니다.
    
    Args:
        df: 데이터프레임
        threshold: 유사도 임계값 (0~1 사이, 1에 가까울수록 더 유사함)
    """
    shop_names = df['상점명'].unique()
    similar_pairs = []
    
    for i in range(len(shop_names)):
        for j in range(i + 1, len(shop_names)):
            similarity = calculate_similarity(shop_names[i], shop_names[j])
            if similarity >= threshold:
                similar_pairs.append({
                    '상점명1': shop_names[i],
                    '상점명2': shop_names[j],
                    '유사도': similarity
                })
    
    return pd.DataFrame(similar_pairs).sort_values('유사도', ascending=False)

In [10]:
# 유사한 상점명 쌍 찾기 (유사도 70% 이상)
similar_shops = find_similar_shop_names(train_data, threshold=0.7)

if not similar_shops.empty:
    print("유사한 상점명 쌍 발견:")
    print(similar_shops.to_string(index=False))
else:
    print("유사도 70% 이상인 상점명 쌍이 없습니다.")

유사한 상점명 쌍 발견:
                                 상점명1                                            상점명2      유사도
           Жуковский ул. Чкалова 39м?                      Жуковский ул. Чкалова 39м² 0.961538
     Москва ТК "Буденовский" (пав.К7)                Москва ТК "Буденовский" (пав.А2) 0.937500
        !Якутск ТЦ "Центральный" фран                         Якутск ТЦ "Центральный" 0.884615
        !Якутск Орджоникидзе, 56 фран                         Якутск Орджоникидзе, 56 0.884615
РостовНаДону ТРК "Мегацентр Горизонт" РостовНаДону ТРК "Мегацентр Горизонт" Островной 0.880952
              Москва ТЦ "Семеновский"                          Москва ТЦ "Перловский" 0.844444
                      Химки ТЦ "Мега"                                  Омск ТЦ "Мега" 0.827586
      Москва ТЦ "МЕГА Теплый Стан" II                  Москва ТЦ "МЕГА Белая Дача II" 0.754098
                 Уфа ТК "Центральный"                         Якутск ТЦ "Центральный" 0.744186
              Казань ТЦ "ПаркХаус" I

In [11]:
# 완전히 동일한 상점들의 ID 매핑 확인
print("=== 동일 상점 ID 확인 ===")
for shop_pair in [
    ('Жуковский ул. Чкалова 39м?', 'Жуковский ул. Чкалова 39м²'),
    ('!Якутск ТЦ "Центральный" фран', 'Якутск ТЦ "Центральный"'),
    ('!Якутск Орджоникидзе, 56 фран', 'Якутск Орджоникидзе, 56')
]:
    shop1_id = train_data[train_data['상점명'] == shop_pair[0]]['상점ID'].iloc[0]
    shop2_id = train_data[train_data['상점명'] == shop_pair[1]]['상점ID'].iloc[0]
    print(f"\n{shop_pair[0]} (ID: {shop1_id})")
    print(f"{shop_pair[1]} (ID: {shop2_id})")

=== 동일 상점 ID 확인 ===

Жуковский ул. Чкалова 39м? (ID: 10)
Жуковский ул. Чкалова 39м² (ID: 11)

!Якутск ТЦ "Центральный" фран (ID: 1)
Якутск ТЦ "Центральный" (ID: 58)

!Якутск Орджоникидзе, 56 фран (ID: 0)
Якутск Орджоникидзе, 56 (ID: 57)


In [ ]:
def unify_shop_ids(df):
    """
    동일 상점에 대해 상점 ID 및 상점명을 통일합니다.
    
    Args:
        df (pd.DataFrame): 상점 데이터가 포함된 데이터프레임
        
    Returns:
        pd.DataFrame: 상점 ID가 통일된 데이터프레임
    """
    id_mapping = {
        0: 57,   # Жуковский ул. Чкалова 39м? -> Жуковский ул. Чкалова 39м²
        1: 58,   # !Якутск ТЦ "Центральный" фран -> Якутск ТЦ "Центральный"
        2: 59    # !Якутск Орджоникидзе, 56 фран -> Якутск Орджоникидзе, 56
    }
    
    for old_id, new_id in id_mapping.items():
        df.loc[df['상점ID'] == old_id, '상점ID'] = new_id
        new_name = df.loc[df['상점ID'] == new_id, '상점명'].iloc[0]
        df.loc[df['상점ID'] == new_id, '상점명'] = new_name

    
    return df

In [13]:
train_data = unify_shop_ids(train_data)
test_data = unify_shop_ids(test_data)   

In [14]:
np.sort(train_data["상점ID"].unique())

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59], dtype=int8)

### 월별 데이터 리셈플링

In [15]:
# 1. 그룹화 및 집계 규칙 정의 (판매가에 'mean'만 적용)
agg_funcs = {
    '판매량': 'sum',
    '판매가': 'mean',  # 표준편차('std') 제거
    '상점명': 'first',
    '상품명': 'first',
    '상품분류ID': 'first',
    '상품분류명': 'first'
}

# 2. GroupBy 및 집계 연산 수행
monthly_data = train_data.groupby(['월ID', '상점ID', '상품ID']).agg(agg_funcs)

# 3. '판매가' 열의 이름만 '판매가_평균'으로 변경
monthly_data.rename(columns={'판매가': '판매가_평균'}, inplace=True)

# 4. 그룹 키를 다시 컬럼으로 변환
monthly_data = monthly_data.reset_index()

# 결과 확인
print("✅ 월별 데이터 집계 완료 (단순화된 방식)")
monthly_data.head(30)

✅ 월별 데이터 집계 완료 (단순화된 방식)


,월ID,상점ID,상품ID,판매량,판매가_평균,상점명,상품명,상품분류ID,상품분류명
0,0,3,32,3,349.0,"Балашиха ТРК ""Октябрь-Киномир""",1+1,40,Кино - DVD
1,0,3,98,5,399.0,"Балашиха ТРК ""Октябрь-Киномир""",1812: УЛАНСКАЯ БАЛЛАДА,40,Кино - DVD
2,0,3,1037,1,3500.0,"Балашиха ТРК ""Октябрь-Киномир""",3D GARDEN (Наш Сад 10) (box),75,Программы - Для дома и офиса
3,0,3,1143,1,399.0,"Балашиха ТРК ""Октябрь-Киномир""",ABBA The Definitive Collection 2CD,55,Музыка - CD локального производства
4,0,3,1201,1,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Back In Black,55,Музыка - CD локального производства
5,0,3,1204,2,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Black Ice,55,Музыка - CD локального производства
6,0,3,1217,2,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Iron Man 2,55,Музыка - CD локального производства
7,0,3,1222,1,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Live,55,Музыка - CD локального производства
8,0,3,1224,1,399.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC Live At River Plate 2CD,55,Музыка - CD локального производства
9,0,3,1235,1,299.0,"Балашиха ТРК ""Октябрь-Киномир""",AC/DC The Rasors Edge,55,Музыка - CD локального производства


In [16]:
test_data.head()

,ID,상점ID,상품ID,상점명,상품명,상품분류ID,상품분류명,월ID
0,0,5,5037,"Вологда ТРЦ ""Мармелад""","NHL 15 [PS3, русские субтитры]",19,Игры - PS3,34
1,1,5,5320,"Вологда ТРЦ ""Мармелад""",ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,34
2,2,5,5233,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,34
3,3,5,5232,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,34
4,4,5,5268,"Вологда ТРЦ ""Мармелад""","Need for Speed [PS4, русская версия]",20,Игры - PS4,34
